http://localhost:8889/notebooks/git/google-quest-challenge/working/QstBertBase0113_1.ipynb

no freezing of bert model

# start

In [1]:
trntmstmp=1584412344
valtmstmp=1585069785
import datetime
print([datetime.datetime.fromtimestamp(o).strftime('%Y-%m-%d %H:%M:%S') for o in (trntmstmp, valtmstmp)])

grand_total=1.5e8
MIN_TM_TRN=1580947200
MIN_TM_TST=1581552000
print([datetime.datetime.fromtimestamp(o).strftime('%Y-%m-%d %H:%M:%S') for o in (MIN_TM_TRN, MIN_TM_TST)])


CHNKSZ=1e6
POST_RATE_WANTED=0.1

['2020-03-17 02:32:24', '2020-03-24 17:09:45']
['2020-02-06 00:00:00', '2020-02-13 00:00:00']


In [2]:
!nvidia-smi

Thu Mar 26 20:47:23 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 418.56       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   78C    P0    76W / 149W |      0MiB / 11441MiB |     97%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# setup

In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
import datetime
def dtnow(): return datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

SEED=101
HOME='/data/git/recsys20'
p_in=f'{HOME}/input'

import torch
from transformers import *
import torch
device=torch.device('cuda')
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader


pretrained_weights='bert-base-multilingual-cased'
bertmodel = BertModel.from_pretrained(pretrained_weights, output_hidden_states=True)
tokenizer = BertTokenizer.from_pretrained(pretrained_weights, do_lower_case=False)


In [4]:
cols=[
'toks',
'hshtgs',
'twtid',
'media',
'links',
'domns',
'twttyp',
'lang',
'tm',

'u1id',
'u1_fllwer_cnt',
'u1_fllwng_cnt',
'u1_vrfed',
'u1_create_tm',

'u2id',
'u2_fllwer_cnt',
'u2_fllwng_cnt',
'u2_vrfed',
'u2_create_tm',

'u1_fllw_u2',
'reply_tm',
'retwt_tm',
'retwt_cmmnt_tm',
'like_tm',
]
cols_cat = ['twttyp','lang']
cols_val = cols[:-4]
cols_tgt_tmstmp=[
    'retwt_tm',
    'reply_tm',
    'like_tm',
    'retwt_cmmnt_tm',
]
cols_tgt=[o.split('_tm')[0] for o in cols_tgt_tmstmp]
tgts             = ['Retweet','Reply','Like','RTwCmnt',]
assert cols_tgt == ['retwt',  'reply','like','retwt_cmmnt',]
ntgts=len(tgts)


tgt2col=dict(zip(tgts,cols_tgt))
tgt2col

{'Retweet': 'retwt',
 'Reply': 'reply',
 'Like': 'like',
 'RTwCmnt': 'retwt_cmmnt'}

# prep

In [5]:
%%time
dftrn=pd.read_csv(
    f'{p_in}/trn_{trntmstmp}.tsv',
    sep='\x01', header=None, 
    encoding='utf-8', 
    names=cols, 
    nrows=1e5
)

CPU times: user 657 ms, sys: 63.9 ms, total: 721 ms
Wall time: 720 ms


In [6]:
# tokenizer.pad_token, tokenizer.pad_token_id
# ('[PAD]', 0)

# tokenizer.sep_token, tokenizer.sep_token_id
# ('[SEP]', 102)
maxlen=512
def mkids(x):
    tokids=list(map(int, x.split('\t')))
    l=len(tokids) 
    if l<=maxlen: 
        return tokids + [0]*(maxlen-len(tokids))
    else: 
        return tokids[:maxlen-1]+[102]

In [7]:
def mk_tensors(df, istrn=True):
    tokids=dftrn.toks.apply(lambda x: mkids(x))
    Xarr=np.array(list(tokids))
    X=torch.tensor(Xarr,dtype=torch.long)
    if not istrn: return X
    ys=dftrn[cols_tgt_tmstmp].notna().values
    ys=torch.tensor(ys,dtype=torch.float)
    return X,ys
    

In [8]:
X,ys = mk_tensors(dftrn)

In [9]:
BS=8
ds = TensorDataset(X,ys)
dl = DataLoader(ds, batch_size=BS, shuffle=True)
for step, batch in enumerate(dl):
    X_b,ys_b = (o.to(device) for o in batch)
    print(X_b.shape,ys_b.shape)
    break

torch.Size([8, 512]) torch.Size([8, 4])


# model

In [10]:
EPOCHS=2
WD=0.01
LR=1e-5
SCHDLR_FUNC = get_cosine_schedule_with_warmup 
WARMUP_RATE = 0.05
N_CYCLS = .5
EPS = 1e-6


In [11]:
BS=6
ds = TensorDataset(X,ys)
dl = DataLoader(ds, batch_size=BS, shuffle=True)
for step, batch in enumerate(dl):
    X_b,ys_b = (o.to(device) for o in batch)
    print(X_b.shape,ys_b.shape)
    break

torch.Size([6, 512]) torch.Size([6, 4])


In [12]:
# https://huggingface.co/transformers/model_doc/bert.html#bertmodel
bertmodel=bertmodel.eval()
bertmodel.to(device)
with torch.no_grad():
    last_hidden_state, pooler_output, hidden_states = bertmodel(X_b,)
    avg_pool = torch.mean(last_hidden_state,1)
    max_pool,_ = torch.max(last_hidden_state,1)
last_hidden_state.shape,pooler_output.shape, len(hidden_states), avg_pool.shape, max_pool.shape

(torch.Size([6, 512, 768]),
 torch.Size([6, 768]),
 13,
 torch.Size([6, 768]),
 torch.Size([6, 768]))

In [13]:
# see RobertaClassificationHead in transformers/modeling_roberta.py
N_HIDDEN = 768 
class TwtModel(nn.Module):
    def __init__(self, bertmodel, num_labels=4):
        super(TwtModel, self).__init__()
        self.bertmodel = bertmodel
#         for param in self.bertmodel.parameters():
#             param.requires_grad=False
        
        dense_size = 64
        self.relu=nn.ReLU(inplace=True)
        nx = N_HIDDEN*2
        self.dense = nn.Linear(nx, nx)
        self.dropout = nn.Dropout(0.1)
        self.out_proj = nn.Linear(nx, num_labels)
        
    def forward(self, x):
        attn_msk = (x!=tokenizer.pad_token_id).float().to(device)
        segments = torch.zeros(x.shape, dtype=torch.long).to(device)
        last_hidden_state, pooler_output, hidden_states = self.bertmodel(x, 
                                                                         attention_mask=attn_msk,
                                         token_type_ids=segments)
        avg_pool = torch.mean(last_hidden_state,1)
        max_pool,_ = torch.max(last_hidden_state,1)
        x = torch.cat([avg_pool,max_pool],1) 
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

In [14]:
model=TwtModel(bertmodel)
model = model.to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': WD},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

optimizer = AdamW(optimizer_grouped_parameters, lr=LR, eps=EPS)
t_total = int(EPOCHS*len(dl))
scheduler = SCHDLR_FUNC(
    optimizer, 
    num_warmup_steps=int(WARMUP_RATE*t_total), 
    num_training_steps=t_total,
    num_cycles=N_CYCLS,
)


In [15]:
for epoch in range(EPOCHS):
    print(dtnow(), f'epoch {epoch} starts')
    y_tr_epoch = []
    y_pred_tr_epoch = []
    for step, batch in enumerate(dl):
        model.train()
        x_b,ys_b = (o.to(device) for o in batch)
        yb_pred = model(x_b)

        loss =  F.binary_cross_entropy_with_logits(yb_pred,ys_b)
        loss.backward()
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        model.zero_grad()
        if step%100==0: print(step, loss.item())


2020-03-26 20:47:40 epoch 0 starts
0 0.714836597442627
100 0.5785242915153503
200 0.37898576259613037
300 0.3492462635040283
400 0.3777930438518524
500 0.2081422209739685
600 0.22332069277763367
700 0.19154012203216553
800 0.20531690120697021
900 0.3143617808818817
1000 0.1878468096256256
1100 0.16255980730056763
1200 0.2713010907173157
1300 0.40963882207870483
1400 0.36319470405578613
1500 0.34884780645370483
1600 0.21671830117702484
1700 0.2313794493675232
1800 0.1792050451040268
1900 0.2604365348815918
2000 0.17257142066955566
2100 0.1810152232646942
2200 0.3823032081127167
2300 0.3344581127166748
2400 0.23860672116279602
2500 0.27880141139030457
2600 0.28563255071640015
2700 0.18617449700832367
2800 0.3656902313232422
2900 0.37608739733695984
3000 0.3149733245372772
3100 0.3387591242790222
3200 0.195344015955925
3300 0.3260330259799957
3400 0.2826628088951111
3500 0.18479816615581512
3600 0.19104349613189697
3700 0.27274203300476074
3800 0.22895443439483643
3900 0.21390706300735474

KeyboardInterrupt: 